In [21]:
#import the necessary packages
from collections import deque
import numpy as np 
import cv2
import time

In [22]:
# will be tracking a yellow ball. Therefore defining the lower and upper boundaries of yellow in the HSV color space and initialize the list of tracked points
yellow_lower = (20, 100, 100)
yellow_upper = (20,255,255)
pts = deque(maxlen=20)

In [23]:
# to capture the video
cap=cv2.VideoCapture(0)

In [24]:
# allow the camera to warm up
time.sleep(2.0)

In [25]:
# create a loop which lasts throughout the time the video is on and works over all frames
while True:
    #capture the current frame
    ret,frame = cap.read()
    
    # blur the frame and convert it to the HSV color space
    blurred = cv2.GaussianBlur(frame(11,11),0)
    hsv = cv2.cvtColor(blurred, cv2.COLOR_BGR2HSV)
    
    # creating a mask for the yellow color by supplying the upper and lower HSV color boundaries, creates a binary mask to identify the ball
    mask = cv2.inRange(hsv,yellow_lower,yellow_upper)
    
    # conductin erosion and dilation to remove any marks left on the ball for two iterations
    mask = cv2.erode(mask,None,iterations = 2)
    mask = cv2.dilate(mask, None, iterations = 2)
    
    # finding the contours in the ball and initialize the center of the ball as none
    contours = cv2.findContours(mask.copy(),cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
    contours = contours[1]
    center = None
    
    # checking to make sure that at least one contour is found
    if len(contours>0):
        #finding the maximum contour and finding the centroid of the minimum enclosing circle of the respective contour
        c = max(contours, key=cv2.contourArea)
        ((x,y),radius) = cv2.minEnclosingCircle(c)
        M = cv2.moments(c)
        center = (int(M["m10"] / M["m00"]), int(M["m01"] / M["m00"]))
        
        # check that the radius is at least of a certain minimum length
        if radius > 10:
            # creating circle around boundary of the ball
            cv2.circle(frame,(int(x),int(y)), int(radius), (0,255,255), 2)
            # indicating the centroid of the ball
            cv2.circle(frame, center, 5, (0,0,255), -1)
    
    # append the centroid to the pts list
    pts.appendleft(center)       
    
    

TypeError: 'numpy.ndarray' object is not callable

In [ ]:
# showing the points at which the ball has previously been tracked
for i in range (1,len(pts)):
    # ignoring any points that are None
    if pts[i-1] is None or pts[i] is None:
        continue
    
    # computing the thickness of the contrail and draw the line
    thickness = int(np.sqrt(20/ float(i+1))*2.5)
    cv2.line(fram, pts[i-1], pts[i], (0,0,255), thickness)
    

In [ ]:
# showing the final output

cv2.imshow("Frame",frame)
key = cv2.waitKey(1) & 0xFF

In [ ]:
# if the key q is pressed the loop will stop

if key == ord("q"):
    sys.exit()

In [26]:
cap.release()
cv2.destroyAllWindows()